In [1]:
import mysql.connector 

connection = mysql.connector.connect(
    host="127.0.0.1",     
    user="root",         
    password="2011DongBaLiem@",
    database="chuanhoacsdl")
cursor = connection.cursor()

In [6]:
cursor.execute(""" 
create table if not exists Customers(
    transaction_id int primary key,
    customer_name varchar(100) not null,
    transaction_date DATE not null)ENGINE=INNODB;  
""")

cursor.execute(""" 
create table if not exists CustomerStocks(
    transaction_id int not null,
    stock_code varchar(10) not null,
    PRIMARY KEY (transaction_id, stock_code),
    FOREIGN KEY(transaction_id) REFERENCES Customers(transaction_id) ) ENGINE = InnoDB;
""")
connection.commit()

In [7]:
try:
    cursor.execute(""" 
    INSERT INTO Customers (transaction_id, customer_name , transaction_date)
    VALUES
        (1,'Nguyen Van A', '2024-12-01'),
        (2, 'Le Thi B', '2024-12-02'),
        (3, 'Tran Van C', '2024-12-03');
    """)
    cursor.execute("""
    INSERT INTO CustomerStocks (transaction_id, stock_code)
    VALUES
        (1,'VNM'), (1,'FPT'), (1,'MWG'),
        (2,'SSI'),
        (3, 'VCB'), (3, 'TCB');
    """)
    connection.commit()
    print("Dữ liệu đã chèn thành công!")

except Exception as e:
    connection.rollback()
    print("Lỗi xảy ra:", e)

Dữ liệu đã chèn thành công!


In [9]:
#Truy vấn danh sách KH và các mã cổ phiếu tương ứng mà họ đã giao dịch
import pandas as pd
query="""
select c.customer_name, c.transaction_date, s.stock_code
FROM Customers c
JOIN CustomerStocks s ON c.transaction_id = s.transaction_id; """
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14444\3031454694.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,customer_name,transaction_date,stock_code
0,Nguyen Van A,2024-12-01,FPT
1,Nguyen Van A,2024-12-01,MWG
2,Nguyen Van A,2024-12-01,VNM
3,Le Thi B,2024-12-02,SSI
4,Tran Van C,2024-12-03,TCB
5,Tran Van C,2024-12-03,VCB


In [10]:
# cập nhật tên khách hàng Nguyen Van A thành Nguyen V. A.
cursor.execute(""" 
UPDATE Customers 
SET customer_name= 'Nguyen V. A'
where 1=1
AND customer_name= 'Nguyen Van A';
""")
connection.commit()

In [11]:
query=""" select * from Customers; """
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14444\3552315168.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,transaction_id,customer_name,transaction_date
0,1,Nguyen V. A,2024-12-01
1,2,Le Thi B,2024-12-02
2,3,Tran Van C,2024-12-03


In [14]:
# Xóa mã cổ phiếu MWG khỏi giao dịch của khách hàng Nguyen V. A

cursor.execute("""
DELETE FROM CustomerStocks 
WHERE transaction_id = (SELECT transaction_id FROM Customers WHERE customer_name = 'Nguyen V. A') 
AND stock_code = 'MWG';
""")
connection.commit()
print("Đã xóa mã MWG khỏi giao dịch của Nguyen V. A.")

Đã xóa mã MWG khỏi giao dịch của Nguyen V. A.


In [15]:
query=""" select * from CustomerStocks; """
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14444\3812986712.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,transaction_id,stock_code
0,1,FPT
1,1,VNM
2,2,SSI
3,3,TCB
4,3,VCB


In [16]:
# Truy vân Đếm số lượng mã cổ phiếu mỗi KH đã giao dịch

query=""" 
SELECT c.customer_name, s.stock_code, count(s.stock_code)
FROM Customers c
JOIN CustomerStocks s ON c.transaction_id = s.transaction_id
GROUP BY c.customer_name, s.stock_code;
"""
connection.commit()
data = pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14444\3852820089.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query,connection)


,customer_name,stock_code,count(s.stock_code)
0,Nguyen V. A,FPT,1
1,Nguyen V. A,VNM,1
2,Le Thi B,SSI,1
3,Tran Van C,TCB,1
4,Tran Van C,VCB,1


In [17]:
#Truy vấn tìm KH có giao dịch nhiều mã cổ phiếu nhất

query="""
SELECT c.customer_name, count(s.stock_code) as count_stock
FROM Customers c
JOIN CustomerStocks s ON c.transaction_id = s.transaction_id
GROUP BY c.customer_name
ORDER BY count_stock DESC
"""
connection.commit()
data = pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14444\1479006604.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query,connection)


,customer_name,count_stock
0,Nguyen V. A,2
1,Tran Van C,2
2,Le Thi B,1
